# Árboles de Clasificación y Bosques Aleatorios
## Machine Learning - Encuentro sobre Modelos de Ensemble

---

## Contenido

1. [Introducción y Conceptos Fundamentales](#introduccion)
2. [Árboles de Decisión para Clasificación](#arboles-decision)
3. [Bosques Aleatorios (Random Forest)](#bosques-aleatorios)
4. [Aplicaciones Prácticas en Ingeniería Electrónica](#aplicaciones)
5. [Métricas de Evaluación](#metricas)
6. [Optimización de Hiperparámetros](#optimizacion)
7. [Comparación con Otros Algoritmos](#comparacion)
8. [Resumen y Conclusiones](#resumen)
Árboles de Clasificación y Bosques Aleatorios  
---

<a id="introduccion"></a>
## 1. ¿Qué son los Árboles de Decisión?

### Concepto Simple
Los **árboles de decisión** son algoritmos que aprenden reglas simples basadas en las características de los datos para clasificar o predecir. Funcionan como un sistema de preguntas que guían hacia una decisión.

### Analogía Visual
Imagine que necesita diagnosticar una falla en un circuito electrónico. El árbol le pregunta:
1. ¿El voltaje está dentro del rango normal? → Sí
2. ¿La corriente supera el umbral seguro? → No  
3. ¿La temperatura del componente es elevada? → Sí
4. **Decisión**: Componente con degradación térmica

### Estructura de un Árbol
![](arbol_decision.svg){width="400"}

---

## 2. Los 4 Conceptos Fundamentales

### 2.1 Nodos y Ramas
- **Nodo Raíz**: La primera pregunta/división
- **Nodos Internos**: Preguntas intermedias
- **Nodos Hoja**: Decisiones finales (clases)
- **Ramas**: Conexiones entre nodos basadas en condiciones

### 2.2 Impureza (Gini / Entropía)
- **Qué es**: Mide qué tan "mezcladas" están las clases en un nodo
- **Objetivo**: Minimizar la impureza en cada división
- **Fórmulas**:
  - $\textsf{Gini} = 1 - \sum_{i=1}^{c} p_i^2$ (donde $p_i$ es la proporción de clase $i$)
  - $\textsf{Entropía} = -\sum_{i=1}^{c} p_i \log_2(p_i)$
- **Valores**:
  - **0**: Nodo puro (una sola clase)
  - **1 (Gini) o log(c) (Entropía)**: Máxima mezcla de clases

### 2.3 Ganancia de Información
- **Qué es**: Diferencia de impureza antes y después de una división
- **Objetivo**: Maximizar la ganancia para elegir la mejor división
- **Fórmula**: $\textsf{Ganancia} = \textsf{Impureza}_\textsf{padre} - \sum_\textsf{hijos} \frac{n_\textsf{hijo}}{n_\textsf{padre}} \times \textsf{Impureza}_\textsf{hijo}$
- **Resultado**: El atributo con mayor ganancia se elige para dividir

### 2.4 Poda (Pruning)
- **Qué es**: Reducir la complejidad del árbol eliminando ramas poco útiles
- **Tipos**:
  - **Poda preventiva**: Limitar profundidad durante construcción
  - **Poda posterior**: Eliminar ramas después de construir el árbol completo
- **Ventaja**: Reduce sobreajuste y mejora generalización

---

## 3. Parámetros Principales

### Parámetro max_depth (Profundidad Máxima)
| Valor             | Significado                   | Uso Recomendado         |
|-------------------|-------------------------------|-------------------------|
| max_depth alto    | Árbol complejo, muchas reglas | Riesgo de sobreajuste   |
| max_depth bajo    | Árbol simple, pocas reglas    | Mejor generalización    |
| **Valor inicial** | max_depth=None (sin límite)   | Ajustar según necesidad |

### Parámetro min_samples_split (Mínimo de Muestras para Dividir)
| Valor                  | Significado                    | Uso Recomendado      |
|------------------------|--------------------------------|----------------------|
| min_samples_split bajo | Más divisiones, árbol complejo | Puede sobreajustar   |
| min_samples_split alto | Menos divisiones, árbol simple | Mejor generalización |
| **Valor inicial**      | min_samples_split=2            | Valor estándar       |

### Parámetro min_samples_leaf (Mínimo de Muestras por Hoja)
| Valor              | Significado                | Uso Recomendado       |
|--------------------|----------------------------|-----------------------|
| min_samples_leaf=1 | Hojas con una sola muestra | Riesgo de sobreajuste |
| min_samples_leaf=5 | Hojas más robustas         | Mejor generalización  |
| **Valor inicial**  | min_samples_leaf=1         | Ajustar según dataset |

---

## 4. ¿Cuándo Usar Árboles de Decisión?

### Ventajas
- **Altamente interpretable**: Reglas claras y fáciles de entender
- **No requiere normalización**: Funciona con datos en diferentes escalas
- **Maneja datos no lineales**: Captura relaciones complejas
- **Detecta variables importantes**: Automáticamente identifica características relevantes
- **Maneja valores faltantes**: Puede trabajar con datos incompletos

### Desventajas
- **Sensible a pequeños cambios**: Pequeños cambios en datos pueden cambiar el árbol
- **Propenso a sobreajuste**: Puede memorizar datos de entrenamiento
- **Puede ser inestable**: Un solo cambio puede cambiar completamente la estructura
- **Sesgo hacia atributos con más valores**: Puede favorecer variables con muchas categorías

### Cuándo Usar
- Necesitas interpretabilidad y explicabilidad
- Trabajas con datos no lineales
- Quieres identificar variables importantes
- Dataset pequeño a mediano
- Necesitas reglas de decisión claras

---


<a id="arboles-decision"></a>
## 5. Árboles de Decisión para Clasificación [⬆](#introduccion)

### Ejemplo Práctico: Clasificación de Componentes Electrónicos

Vamos a trabajar con un ejemplo realista: **clasificar el estado de integridad de componentes electrónicos** basándonos en mediciones físicas y eléctricas.

**Características**:
- Voltaje de operación (V)
- Corriente de consumo (mA)
- Temperatura de operación (°C)
- Factor de potencia
- Frecuencia de resonancia (kHz)

**Clases**:
- **0**: Componente funcional
- **1**: Componente con degradación leve
- **2**: Componente con falla crítica


In [ ]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix,
                            ConfusionMatrixDisplay, roc_curve, auc)
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Librerías importadas correctamente")


In [ ]:
# Generar dataset sintético: Componentes Electrónicos
np.random.seed(42)
n_muestras = 500

# Generar características relacionadas con el estado del componente
# Componentes funcionales (clase 0): parámetros normales
n_funcionales = 200
voltaje_0 = np.random.normal(5.0, 0.3, n_funcionales)  # Voltaje estable alrededor de 5V
corriente_0 = np.random.normal(100, 10, n_funcionales)  # Corriente normal ~100mA
temperatura_0 = np.random.normal(45, 5, n_funcionales)  # Temperatura normal ~45°C
factor_potencia_0 = np.random.normal(0.95, 0.03, n_funcionales)  # Factor de potencia alto
frecuencia_0 = np.random.normal(1000, 50, n_funcionales)  # Frecuencia de resonancia normal

# Componentes con degradación leve (clase 1): algunos parámetros fuera de rango
n_degradados = 200
voltaje_1 = np.random.normal(5.5, 0.5, n_degradados)  # Voltaje ligeramente elevado
corriente_1 = np.random.normal(120, 15, n_degradados)  # Corriente aumentada
temperatura_1 = np.random.normal(60, 8, n_degradados)  # Temperatura elevada
factor_potencia_1 = np.random.normal(0.75, 0.1, n_degradados)  # Factor de potencia bajo
frecuencia_1 = np.random.normal(950, 80, n_degradados)  # Frecuencia desviada

# Componentes con falla crítica (clase 2): parámetros muy fuera de rango
n_fallados = 100
voltaje_2 = np.random.normal(6.5, 1.0, n_fallados)  # Voltaje muy alto
corriente_2 = np.random.normal(180, 30, n_fallados)  # Corriente muy alta
temperatura_2 = np.random.normal(85, 15, n_fallados)  # Temperatura crítica
factor_potencia_2 = np.random.normal(0.5, 0.15, n_fallados)  # Factor de potencia muy bajo
frecuencia_2 = np.random.normal(800, 100, n_fallados)  # Frecuencia muy desviada

# Combinar todos los datos
voltaje = np.concatenate([voltaje_0, voltaje_1, voltaje_2])
corriente = np.concatenate([corriente_0, corriente_1, corriente_2])
temperatura = np.concatenate([temperatura_0, temperatura_1, temperatura_2])
factor_potencia = np.concatenate([factor_potencia_0, factor_potencia_1, factor_potencia_2])
frecuencia = np.concatenate([frecuencia_0, frecuencia_1, frecuencia_2])

# Crear etiquetas: 0=funcional, 1=degradado, 2=fallado
etiquetas = np.concatenate([
    np.zeros(n_funcionales, dtype=int),
    np.ones(n_degradados, dtype=int),
    np.full(n_fallados, 2, dtype=int)
])

# Crear DataFrame
df_componentes = pd.DataFrame({
    'voltaje': voltaje,
    'corriente': corriente,
    'temperatura': temperatura,
    'factor_potencia': factor_potencia,
    'frecuencia': frecuencia,
    'estado': etiquetas
})

print("Dataset generado:")
print(f"Muestras totales: {len(df_componentes)}")
print(f"\nDistribución por clase:")
print(df_componentes['estado'].value_counts().sort_index())
print(f"\nPrimeras 10 muestras:")
print(df_componentes.head(10))


In [ ]:
# Visualización exploratoria de los datos
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Mapeo de colores para las clases
colores = {0: 'green', 1: 'orange', 2: 'red'}
nombres_clases = {0: 'Funcional', 1: 'Degradado', 2: 'Falla Crítica'}

# Gráfico 1: Voltaje vs Corriente
ax = axes[0, 0]
for clase in [0, 1, 2]:
    mask = df_componentes['estado'] == clase
    ax.scatter(df_componentes[mask]['voltaje'], df_componentes[mask]['corriente'],
               c=colores[clase], label=nombres_clases[clase], alpha=0.6, s=50)
ax.set_xlabel('Voltaje (V)')
ax.set_ylabel('Corriente (mA)')
ax.set_title('Voltaje vs Corriente por Estado')
ax.legend()
ax.grid(True, alpha=0.3)

# Gráfico 2: Temperatura vs Factor de Potencia
ax = axes[0, 1]
for clase in [0, 1, 2]:
    mask = df_componentes['estado'] == clase
    ax.scatter(df_componentes[mask]['temperatura'], df_componentes[mask]['factor_potencia'],
               c=colores[clase], label=nombres_clases[clase], alpha=0.6, s=50)
ax.set_xlabel('Temperatura (°C)')
ax.set_ylabel('Factor de Potencia')
ax.set_title('Temperatura vs Factor de Potencia')
ax.legend()
ax.grid(True, alpha=0.3)

# Gráfico 3: Frecuencia por Estado
ax = axes[0, 2]
df_componentes.boxplot(column='frecuencia', by='estado', ax=ax)
ax.set_xlabel('Estado del Componente')
ax.set_ylabel('Frecuencia (kHz)')
ax.set_title('Distribución de Frecuencia por Estado')
ax.set_xticklabels(['Funcional', 'Degradado', 'Falla Crítica'])
plt.suptitle('')  # Eliminar título automático

# Gráfico 4: Matriz de correlación
ax = axes[1, 0]
correlation_matrix = df_componentes.iloc[:, :-1].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.2f', ax=ax)
ax.set_title('Matriz de Correlación entre Características')

# Gráfico 5: Distribución de Temperatura
ax = axes[1, 1]
for clase in [0, 1, 2]:
    mask = df_componentes['estado'] == clase
    ax.hist(df_componentes[mask]['temperatura'], bins=20, alpha=0.6,
            label=nombres_clases[clase], color=colores[clase])
ax.set_xlabel('Temperatura (°C)')
ax.set_ylabel('Frecuencia')
ax.set_title('Distribución de Temperatura por Estado')
ax.legend()
ax.grid(True, alpha=0.3)

# Gráfico 6: Distribución de Voltaje
ax = axes[1, 2]
for clase in [0, 1, 2]:
    mask = df_componentes['estado'] == clase
    ax.hist(df_componentes[mask]['voltaje'], bins=20, alpha=0.6,
            label=nombres_clases[clase], color=colores[clase])
ax.set_xlabel('Voltaje (V)')
ax.set_ylabel('Frecuencia')
ax.set_title('Distribución de Voltaje por Estado')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Dividir datos en entrenamiento y prueba
X = df_componentes[['voltaje', 'corriente', 'temperatura', 'factor_potencia', 'frecuencia']]
y = df_componentes['estado']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Entrenamiento: {len(X_train)} muestras")
print(f"Prueba: {len(X_test)} muestras")
print(f"\nDistribución en entrenamiento:")
print(y_train.value_counts().sort_index())
print(f"\nDistribución en prueba:")
print(y_test.value_counts().sort_index())


In [ ]:
# Entrenar árbol de decisión básico (sin restricciones)
arbol_basico = DecisionTreeClassifier(random_state=42)
arbol_basico.fit(X_train, y_train)

# Predecir
y_pred_basico = arbol_basico.predict(X_test)

# Calcular métricas
accuracy_basico = accuracy_score(y_test, y_pred_basico)

print("=" * 60)
print("ÁRBOL DE DECISIÓN BÁSICO (sin restricciones)")
print("=" * 60)
print(f"Precisión (Accuracy): {accuracy_basico:.4f}")
print(f"Profundidad del árbol: {arbol_basico.get_depth()}")
print(f"Número de nodos hoja: {arbol_basico.get_n_leaves()}")
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred_basico,
                           target_names=['Funcional', 'Degradado', 'Falla Crítica']))

# Matriz de confusión
cm_basico = confusion_matrix(y_test, y_pred_basico)
print("\nMatriz de confusión:")
print(cm_basico)


In [ ]:
# Visualizar el árbol de decisión (primeros niveles para legibilidad)
plt.figure(figsize=(20, 12))
plot_tree(arbol_basico,
          feature_names=['Voltaje', 'Corriente', 'Temperatura', 'Factor Potencia', 'Frecuencia'],
          class_names=['Funcional', 'Degradado', 'Falla Crítica'],
          filled=True,
          rounded=True,
          fontsize=10,
          max_depth=3)  # Mostrar solo primeros 3 niveles
plt.title("Árbol de Decisión (Primeros 3 niveles)", fontsize=16, pad=20)
plt.show()


In [ ]:
# Imprimir las reglas de decisión en texto (primeros niveles)
reglas_texto = export_text(arbol_basico,
                           feature_names=['Voltaje', 'Corriente', 'Temperatura', 'Factor Potencia', 'Frecuencia'],
                           max_depth=3,
                           decimals=2)
print("Reglas de Decisión (Primeros 3 niveles):")
print("=" * 60)
print(reglas_texto)


In [ ]:
# Comparar árboles con diferentes profundidades máximas
profundidades = [3, 5, 7, 10, 15, None]
resultados_profundidad = []

for max_d in profundidades:
    arbol = DecisionTreeClassifier(max_depth=max_d, random_state=42)
    arbol.fit(X_train, y_train)

    # Predecir
    y_pred_train = arbol.predict(X_train)
    y_pred_test = arbol.predict(X_test)

    # Calcular métricas
    acc_train = accuracy_score(y_train, y_pred_train)
    acc_test = accuracy_score(y_test, y_pred_test)

    resultados_profundidad.append({
        'max_depth': max_d if max_d is not None else 'Sin límite',
        'profundidad_real': arbol.get_depth(),
        'nodos_hoja': arbol.get_n_leaves(),
        'acc_train': acc_train,
        'acc_test': acc_test
    })

df_profundidades = pd.DataFrame(resultados_profundidad)
print("Comparación de Árboles con Diferentes Profundidades:")
print("=" * 70)
print(df_profundidades.to_string(index=False))

# Visualizar resultados
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Gráfico 1: Accuracy vs Profundidad
ax = axes[0]
x_pos = range(len(df_profundidades))
ax.plot(x_pos, df_profundidades['acc_train'], 'o-', label='Entrenamiento', linewidth=2, markersize=8)
ax.plot(x_pos, df_profundidades['acc_test'], 's-', label='Prueba', linewidth=2, markersize=8)
ax.set_xticks(x_pos)
ax.set_xticklabels(df_profundidades['max_depth'], rotation=45)
ax.set_xlabel('Profundidad Máxima')
ax.set_ylabel('Accuracy')
ax.set_title('Accuracy vs Profundidad Máxima')
ax.legend()
ax.grid(True, alpha=0.3)

# Gráfico 2: Número de Nodos Hoja vs Profundidad
ax = axes[1]
ax.bar(x_pos, df_profundidades['nodos_hoja'], alpha=0.7, color='skyblue')
ax.set_xticks(x_pos)
ax.set_xticklabels(df_profundidades['max_depth'], rotation=45)
ax.set_xlabel('Profundidad Máxima')
ax.set_ylabel('Número de Nodos Hoja')
ax.set_title('Complejidad del Árbol')
ax.grid(True, alpha=0.3, axis='y')

# Gráfico 3: Diferencia entre Train y Test (sobreajuste)
ax = axes[2]
diferencia = df_profundidades['acc_train'] - df_profundidades['acc_test']
ax.bar(x_pos, diferencia, alpha=0.7, color='coral')
ax.set_xticks(x_pos)
ax.set_xticklabels(df_profundidades['max_depth'], rotation=45)
ax.set_xlabel('Profundidad Máxima')
ax.set_ylabel('Diferencia (Train - Test)')
ax.set_title('Indicador de Sobreajuste')
ax.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Identificar mejor árbol (balance entre accuracy y generalización)
# Penalizar sobreajuste: score = acc_test - 0.5 * (acc_train - acc_test)
df_profundidades['score_balanceado'] = (df_profundidades['acc_test'] -
                                        0.5 * (df_profundidades['acc_train'] - df_profundidades['acc_test']))
mejor_idx = df_profundidades['score_balanceado'].idxmax()
mejor_profundidad = df_profundidades.loc[mejor_idx, 'max_depth']

print(f"\nMejor profundidad (balance entre accuracy y generalización): {mejor_profundidad}")
print(f"Accuracy en prueba: {df_profundidades.loc[mejor_idx, 'acc_test']:.4f}")


<a id="bosques-aleatorios"></a>
## 6. Bosques Aleatorios (Random Forest) [⬆](#introduccion)

### ¿Qué es un Bosque Aleatorio?

Un **Bosque Aleatorio** es un algoritmo de **ensemble** (conjunto) que combina múltiples árboles de decisión para mejorar la precisión y reducir el sobreajuste.

### Concepto Simple
En lugar de confiar en un solo árbol, entrenamos **cientos de árboles** y tomamos la decisión por **votación mayoritaria**.

### Analogía
Imagine que necesita diagnosticar una falla. En lugar de preguntar a un solo experto:
- Árbol de Decisión: Un experto hace todas las preguntas
- Bosque Aleatorio: 100 expertos, cada uno hace preguntas diferentes, y votamos por la respuesta más común

### Los 3 Conceptos Fundamentales

#### 6.1 Bootstrap Aggregating (Bagging)
- **Qué es**: Cada árbol se entrena con una **muestra aleatoria** de los datos (con reemplazo)
- **Ventaja**: Cada árbol ve datos ligeramente diferentes, reduciendo varianza

#### 6.2 Selección Aleatoria de Características
- **Qué es**: En cada división, solo se consideran un **subconjunto aleatorio** de características
- **Valor típico**: $\sqrt{n}$ características para clasificación (donde $n$ es el total de características)
- **Ventaja**: Reduce correlación entre árboles y mejora diversidad

#### 6.3 Votación Mayoritaria
- **Qué es**: La clase predicha es la que más árboles votaron
- **Para probabilidades**: Promedio de probabilidades de todos los árboles

### Parámetros Principales

| Parámetro           | Descripción                          | Valor Recomendado      |
|---------------------|--------------------------------------|------------------------|
| **n_estimators**    | Número de árboles en el bosque       | 100-200 (más = mejor, pero más lento) |
| **max_depth**       | Profundidad máxima de cada árbol     | None o 10-20           |
| **min_samples_split** | Mínimo muestras para dividir      | 2 o más               |
| **min_samples_leaf**  | Mínimo muestras por hoja            | 1 o más               |
| **max_features**    | Número de características por división | 'sqrt' (clasificación) |

### Ventajas sobre Árboles Individuales
- **Menos sobreajuste**: Promediar múltiples árboles reduce varianza
- **Mayor precisión**: Generalmente mejor rendimiento
- **Robusto a outliers**: Diferentes árboles compensan errores
- **Importancia de características**: Mide qué características son más relevantes
- **Maneja datos desbalanceados**: Puede balancear clases

### Desventajas
- **Menos interpretable**: No puedes "ver" un solo árbol fácilmente
- **Más lento**: Requiere entrenar múltiples árboles
- **Más memoria**: Almacena múltiples modelos

---


In [ ]:
# Importar Random Forest
from sklearn.ensemble import RandomForestClassifier

# Entrenar un Bosque Aleatorio básico
bosque_basico = RandomForestClassifier(n_estimators=100, random_state=42)
bosque_basico.fit(X_train, y_train)

# Predecir
y_pred_bosque = bosque_basico.predict(X_test)

# Calcular métricas
accuracy_bosque = accuracy_score(y_test, y_pred_bosque)

print("=" * 60)
print("BOSQUE ALEATORIO BÁSICO (100 árboles)")
print("=" * 60)
print(f"Precisión (Accuracy): {accuracy_bosque:.4f}")
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred_bosque,
                           target_names=['Funcional', 'Degradado', 'Falla Crítica']))

# Matriz de confusión
cm_bosque = confusion_matrix(y_test, y_pred_bosque)
print("\nMatriz de confusión:")
print(cm_bosque)

# Comparar con árbol individual
print("\n" + "=" * 60)
print("COMPARACIÓN: Árbol Individual vs Bosque Aleatorio")
print("=" * 60)
print(f"Árbol Individual (sin límite):     {accuracy_basico:.4f}")
print(f"Bosque Aleatorio (100 árboles):    {accuracy_bosque:.4f}")
print(f"Mejora:                            {accuracy_bosque - accuracy_basico:.4f}")

In [ ]:
# Visualizar importancia de características (Feature Importance)
importancias = bosque_basico.feature_importances_
caracteristicas = ['Voltaje', 'Corriente', 'Temperatura', 'Factor Potencia', 'Frecuencia']

# Crear DataFrame para mejor visualización
df_importancias = pd.DataFrame({
    'Característica': caracteristicas,
    'Importancia': importancias
}).sort_values('Importancia', ascending=False)

print("Importancia de Características (Feature Importance):")
print("=" * 50)
for idx, row in df_importancias.iterrows():
    print(f"{row['Característica']:20} : {row['Importancia']:.4f} ({row['Importancia']*100:.2f}%)")

# Visualizar importancia
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico de barras horizontal
ax = axes[0]
colores_barras = plt.cm.viridis(df_importancias['Importancia'] / df_importancias['Importancia'].max())
ax.barh(df_importancias['Característica'], df_importancias['Importancia'], color=colores_barras)
ax.set_xlabel('Importancia')
ax.set_title('Importancia de Características - Bosque Aleatorio')
ax.grid(True, alpha=0.3, axis='x')

# Gráfico de pastel
ax = axes[1]
ax.pie(df_importancias['Importancia'], labels=df_importancias['Característica'],
       autopct='%1.1f%%', startangle=90)
ax.set_title('Distribución de Importancia')

plt.tight_layout()
plt.show()


In [ ]:
# Comparar diferentes números de árboles (n_estimators)
n_arboles_lista = [10, 25, 50, 100, 200, 500]
resultados_n_arboles = []

for n_arboles in n_arboles_lista:
    bosque = RandomForestClassifier(n_estimators=n_arboles, random_state=42, n_jobs=-1)
    bosque.fit(X_train, y_train)

    # Predecir
    y_pred_train = bosque.predict(X_train)
    y_pred_test = bosque.predict(X_test)

    # Calcular métricas
    acc_train = accuracy_score(y_train, y_pred_train)
    acc_test = accuracy_score(y_test, y_pred_test)

    # Tiempo de entrenamiento (aproximado)
    import time
    inicio = time.time()
    bosque_temp = RandomForestClassifier(n_estimators=n_arboles, random_state=42, n_jobs=-1)
    bosque_temp.fit(X_train, y_train)
    tiempo = time.time() - inicio

    resultados_n_arboles.append({
        'n_estimators': n_arboles,
        'acc_train': acc_train,
        'acc_test': acc_test,
        'tiempo_seg': tiempo
    })

df_n_arboles = pd.DataFrame(resultados_n_arboles)
print("Comparación de Bosques con Diferentes Números de Árboles:")
print("=" * 70)
print(df_n_arboles.to_string(index=False))

# Visualizar resultados
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico 1: Accuracy vs Número de Árboles
ax = axes[0]
ax.plot(df_n_arboles['n_estimators'], df_n_arboles['acc_train'], 'o-',
        label='Entrenamiento', linewidth=2, markersize=8)
ax.plot(df_n_arboles['n_estimators'], df_n_arboles['acc_test'], 's-',
        label='Prueba', linewidth=2, markersize=8)
ax.set_xlabel('Número de Árboles (n_estimators)')
ax.set_ylabel('Accuracy')
ax.set_title('Accuracy vs Número de Árboles')
ax.legend()
ax.grid(True, alpha=0.3)

# Gráfico 2: Tiempo de Entrenamiento vs Número de Árboles
ax = axes[1]
ax.plot(df_n_arboles['n_estimators'], df_n_arboles['tiempo_seg'], 'ro-',
        linewidth=2, markersize=8)
ax.set_xlabel('Número de Árboles (n_estimators)')
ax.set_ylabel('Tiempo de Entrenamiento (segundos)')
ax.set_title('Costo Computacional')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Visualizar matrices de confusión comparativas
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Matriz de confusión - Árbol Individual
ax = axes[0]
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_basico,
                                       display_labels=['Funcional', 'Degradado', 'Falla Crítica'],
                                       ax=ax, cmap='Blues')
ax.set_title('Árbol de Decisión Individual')
ax.set_xlabel('Predicción')
ax.set_ylabel('Valor Real')

# Matriz de confusión - Bosque Aleatorio
ax = axes[1]
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_bosque,
                                       display_labels=['Funcional', 'Degradado', 'Falla Crítica'],
                                       ax=ax, cmap='Greens')
ax.set_title('Bosque Aleatorio (100 árboles)')
ax.set_xlabel('Predicción')
ax.set_ylabel('Valor Real')

plt.tight_layout()
plt.show()


<a id="aplicaciones"></a>
## 7. Aplicación Práctica: Clasificación de Fallas en Circuitos Integrados [⬆](#introduccion)

Vamos a implementar un ejemplo más complejo: **detección de fallas en circuitos integrados** basándonos en pruebas funcionales.

**Contexto**: En la fabricación de CI, cada chip se prueba con múltiples señales de entrada. Necesitamos clasificar si el chip es:
- **Funcional** (pasa todas las pruebas)
- **Fallado** (falla críticas)
- **Marginal** (pasa pruebas pero con degradación)

**Características**:
- Respuesta a frecuencia alta (MHz)
- Respuesta a frecuencia baja (Hz)
- Consumo de potencia (mW)
- Voltaje de umbral (V)
- Tiempo de respuesta (ns)
- Ganancia (dB)


In [ ]:
# Generar dataset más complejo: Circuitos Integrados
np.random.seed(123)
n_chips = 800

# Chips funcionales (clase 0): 60% de la producción
n_funcionales = int(0.6 * n_chips)
freq_alta_func = np.random.normal(100, 5, n_funcionales)  # MHz, respuesta a alta frecuencia
freq_baja_func = np.random.normal(1, 0.1, n_funcionales)  # Hz, respuesta a baja frecuencia
consumo_func = np.random.normal(50, 5, n_funcionales)     # mW, consumo normal
voltaje_umbral_func = np.random.normal(0.7, 0.05, n_funcionales)  # V, voltaje umbral normal
tiempo_respuesta_func = np.random.normal(10, 1, n_funcionales)    # ns, tiempo de respuesta rápido
ganancia_func = np.random.normal(40, 2, n_funcionales)            # dB, ganancia alta

# Chips marginales (clase 1): 30% de la producción
n_marginales = int(0.3 * n_chips)
freq_alta_marg = np.random.normal(95, 8, n_marginales)      # Frecuencia ligeramente reducida
freq_baja_marg = np.random.normal(1.2, 0.2, n_marginales)  # Desviación en baja frecuencia
consumo_marg = np.random.normal(65, 8, n_marginales)        # Consumo aumentado
voltaje_umbral_marg = np.random.normal(0.75, 0.08, n_marginales)  # Voltaje umbral elevado
tiempo_respuesta_marg = np.random.normal(15, 2, n_marginales)      # Respuesta más lenta
ganancia_marg = np.random.normal(35, 3, n_marginales)               # Ganancia reducida

# Chips fallados (clase 2): 10% de la producción
n_fallados = n_chips - n_funcionales - n_marginales
freq_alta_fall = np.random.normal(80, 15, n_fallados)      # Frecuencia muy reducida
freq_baja_fall = np.random.normal(1.5, 0.5, n_fallados)   # Desviación significativa
consumo_fall = np.random.normal(100, 20, n_fallados)       # Consumo excesivo
voltaje_umbral_fall = np.random.normal(0.9, 0.15, n_fallados)  # Voltaje umbral muy alto
tiempo_respuesta_fall = np.random.normal(30, 5, n_fallados)     # Respuesta muy lenta
ganancia_fall = np.random.normal(25, 5, n_fallados)              # Ganancia muy baja

# Combinar datos
freq_alta = np.concatenate([freq_alta_func, freq_alta_marg, freq_alta_fall])
freq_baja = np.concatenate([freq_baja_func, freq_baja_marg, freq_baja_fall])
consumo = np.concatenate([consumo_func, consumo_marg, consumo_fall])
voltaje_umbral = np.concatenate([voltaje_umbral_func, voltaje_umbral_marg, voltaje_umbral_fall])
tiempo_respuesta = np.concatenate([tiempo_respuesta_func, tiempo_respuesta_marg, tiempo_respuesta_fall])
ganancia = np.concatenate([ganancia_func, ganancia_marg, ganancia_fall])

# Etiquetas: 0=funcional, 1=marginal, 2=fallado
etiquetas_ci = np.concatenate([
    np.zeros(n_funcionales, dtype=int),
    np.ones(n_marginales, dtype=int),
    np.full(n_fallados, 2, dtype=int)
])

# Crear DataFrame
df_chips = pd.DataFrame({
    'freq_alta': freq_alta,
    'freq_baja': freq_baja,
    'consumo': consumo,
    'voltaje_umbral': voltaje_umbral,
    'tiempo_respuesta': tiempo_respuesta,
    'ganancia': ganancia,
    'estado': etiquetas_ci
})

print("Dataset de Circuitos Integrados:")
print(f"Muestras totales: {len(df_chips)}")
print(f"\nDistribución por clase:")
print(df_chips['estado'].value_counts().sort_index())
print("\nEstadísticas descriptivas:")
df_chips.describe()


In [ ]:
# Dividir datos
X_ci = df_chips[['freq_alta', 'freq_baja', 'consumo', 'voltaje_umbral', 'tiempo_respuesta', 'ganancia']]
y_ci = df_chips['estado']

X_train_ci, X_test_ci, y_train_ci, y_test_ci = train_test_split(
    X_ci, y_ci, test_size=0.2, random_state=42, stratify=y_ci
)

print(f"Entrenamiento: {len(X_train_ci)} muestras")
print(f"Prueba: {len(X_test_ci)} muestras")


In [ ]:
# Entrenar múltiples modelos y comparar
modelos_ci = {
    'Árbol (max_depth=5)': DecisionTreeClassifier(max_depth=5, random_state=42),
    'Árbol (max_depth=10)': DecisionTreeClassifier(max_depth=10, random_state=42),
    'Bosque (50 árboles)': RandomForestClassifier(n_estimators=50, random_state=42),
    'Bosque (100 árboles)': RandomForestClassifier(n_estimators=100, random_state=42),
    'Bosque (200 árboles)': RandomForestClassifier(n_estimators=200, random_state=42)
}

resultados_ci = {}

print("Comparación de Modelos para Clasificación de Circuitos Integrados:")
print("=" * 70)

for nombre, modelo in modelos_ci.items():
    modelo.fit(X_train_ci, y_train_ci)
    y_pred_train = modelo.predict(X_train_ci)
    y_pred_test = modelo.predict(X_test_ci)

    acc_train = accuracy_score(y_train_ci, y_pred_train)
    acc_test = accuracy_score(y_test_ci, y_pred_test)

    resultados_ci[nombre] = {
        'acc_train': acc_train,
        'acc_test': acc_test
    }

    print(f"{nombre:25} | Train: {acc_train:.4f} | Test: {acc_test:.4f}")

# Crear DataFrame de resultados
df_resultados_ci = pd.DataFrame(resultados_ci).T
print("\n" + "=" * 70)

# Visualizar comparación
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico de barras: Accuracy
ax = axes[0]
x_pos = range(len(df_resultados_ci))
width = 0.35
ax.bar([x - width/2 for x in x_pos], df_resultados_ci['acc_train'],
       width, label='Entrenamiento', alpha=0.8)
ax.bar([x + width/2 for x in x_pos], df_resultados_ci['acc_test'],
       width, label='Prueba', alpha=0.8)
ax.set_xticks(x_pos)
ax.set_xticklabels(df_resultados_ci.index, rotation=45, ha='right')
ax.set_ylabel('Accuracy')
ax.set_title('Comparación de Modelos')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Gráfico: Diferencia Train-Test (sobreajuste)
ax = axes[1]
diferencia = df_resultados_ci['acc_train'] - df_resultados_ci['acc_test']
ax.bar(x_pos, diferencia, alpha=0.7, color='coral')
ax.set_xticks(x_pos)
ax.set_xticklabels(df_resultados_ci.index, rotation=45, ha='right')
ax.set_ylabel('Diferencia (Train - Test)')
ax.set_title('Indicador de Sobreajuste')
ax.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Seleccionar mejor modelo
mejor_modelo_nombre = df_resultados_ci['acc_test'].idxmax()
mejor_modelo_acc = df_resultados_ci.loc[mejor_modelo_nombre, 'acc_test']
print(f"\nMejor modelo: {mejor_modelo_nombre}")
print(f"Accuracy en prueba: {mejor_modelo_acc:.4f}")


In [ ]:
# Entrenar el mejor modelo y obtener análisis detallado
mejor_modelo_ci = RandomForestClassifier(n_estimators=200, random_state=42)
mejor_modelo_ci.fit(X_train_ci, y_train_ci)
y_pred_ci = mejor_modelo_ci.predict(X_test_ci)

print("=" * 70)
print("ANÁLISIS DETALLADO - BOSQUE ALEATORIO (200 árboles)")
print("=" * 70)
print("\nReporte de clasificación:")
print(classification_report(y_test_ci, y_pred_ci,
                           target_names=['Funcional', 'Marginal', 'Fallado']))

# Matriz de confusión detallada
cm_ci = confusion_matrix(y_test_ci, y_pred_ci)
print("\nMatriz de confusión (valores absolutos):")
print(cm_ci)

# Matriz de confusión en porcentajes
cm_ci_percent = cm_ci.astype('float') / cm_ci.sum(axis=1)[:, np.newaxis]
print("\nMatriz de confusión (porcentajes):")
np.set_printoptions(precision=2, suppress=True)
print(cm_ci_percent)

# Visualizar matriz de confusión
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ConfusionMatrixDisplay.from_predictions(y_test_ci, y_pred_ci,
                                       display_labels=['Funcional', 'Marginal', 'Fallado'],
                                       ax=axes[0], cmap='Blues')
axes[0].set_title('Matriz de Confusión (Valores Absolutos)')

ConfusionMatrixDisplay(confusion_matrix=cm_ci_percent,
                       display_labels=['Funcional', 'Marginal', 'Fallado']).plot(
                       ax=axes[1], cmap='Blues', values_format='.2f')
axes[1].set_title('Matriz de Confusión (Porcentajes)')

plt.tight_layout()
plt.show()

# Importancia de características
importancias_ci = mejor_modelo_ci.feature_importances_
caracteristicas_ci = ['Freq Alta', 'Freq Baja', 'Consumo', 'Voltaje Umbral',
                     'Tiempo Respuesta', 'Ganancia']

df_importancias_ci = pd.DataFrame({
    'Característica': caracteristicas_ci,
    'Importancia': importancias_ci
}).sort_values('Importancia', ascending=False)

print("\nImportancia de Características:")
print("=" * 50)
for idx, row in df_importancias_ci.iterrows():
    print(f"{row['Característica']:20} : {row['Importancia']:.4f} ({row['Importancia']*100:.2f}%)")

# Visualizar importancia
plt.figure(figsize=(10, 6))
colores_ci = plt.cm.viridis(df_importancias_ci['Importancia'] / df_importancias_ci['Importancia'].max())
plt.barh(df_importancias_ci['Característica'], df_importancias_ci['Importancia'],
         color=colores_ci)
plt.xlabel('Importancia')
plt.title('Importancia de Características - Detección de Fallas en CI')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()


<a id="optimizacion"></a>
## 8. Optimización de Hiperparámetros con Grid Search [⬆](#introduccion)

El **Grid Search** prueba sistemáticamente diferentes combinaciones de hiperparámetros para encontrar la mejor configuración.

### Proceso:
1. Definir una **grilla** de valores para cada parámetro
2. Probar **todas las combinaciones**
3. Evaluar con **validación cruzada**
4. Seleccionar la **mejor combinación**


In [ ]:
# Grid Search para optimizar hiperparámetros del Bosque Aleatorio
# Definir grilla de parámetros
param_grid_bosque = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Crear GridSearchCV con validación cruzada (5-fold)
grid_bosque = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid_bosque,
    cv=5,                              # Validación cruzada de 5 folds
    scoring='accuracy',                # Métrica a optimizar
    n_jobs=-1,                         # Usar todos los núcleos disponibles
    verbose=1                          # Mostrar progreso
)

print("Iniciando Grid Search (esto puede tomar unos minutos)...")
print("=" * 70)

# Entrenar con todas las combinaciones
grid_bosque.fit(X_train_ci, y_train_ci)

print("\n" + "=" * 70)
print("RESULTADOS DEL GRID SEARCH")
print("=" * 70)
print(f"\nMejores parámetros encontrados:")
print(grid_bosque.best_params_)
print(f"\nMejor score (accuracy) con validación cruzada: {grid_bosque.best_score_:.4f}")

# Evaluar el mejor modelo en el conjunto de prueba
mejor_bosque = grid_bosque.best_estimator_
y_pred_optimizado = mejor_bosque.predict(X_test_ci)
accuracy_optimizado = accuracy_score(y_test_ci, y_pred_optimizado)

print(f"\nAccuracy en conjunto de prueba: {accuracy_optimizado:.4f}")

# Comparar con modelo sin optimizar
print("\n" + "=" * 70)
print("COMPARACIÓN: Sin Optimizar vs Optimizado")
print("=" * 70)
print(f"Bosque sin optimizar (200 árboles, parámetros por defecto):")
print(f"  Accuracy en prueba: {df_resultados_ci.loc['Bosque (200 árboles)', 'acc_test']:.4f}")
print(f"\nBosque optimizado (Grid Search):")
print(f"  Accuracy en prueba: {accuracy_optimizado:.4f}")
print(f"  Mejora: {accuracy_optimizado - df_resultados_ci.loc['Bosque (200 árboles)', 'acc_test']:.4f}")

# Mostrar algunos de los mejores resultados
print("\nTop 5 combinaciones de parámetros:")
resultados_grid = pd.DataFrame(grid_bosque.cv_results_)
top_5 = resultados_grid.nlargest(5, 'mean_test_score')[
    ['param_n_estimators', 'param_max_depth', 'param_min_samples_split',
     'param_min_samples_leaf', 'mean_test_score', 'std_test_score']
]
print(top_5.to_string(index=False))


In [ ]:
# Validación cruzada para evaluar robustez del modelo
print("=" * 70)
print("VALIDACIÓN CRUZADA (5-FOLD)")
print("=" * 70)

# Realizar validación cruzada con el mejor modelo
cv_scores = cross_val_score(mejor_bosque, X_train_ci, y_train_ci, cv=5, scoring='accuracy')

print(f"Scores por fold: {cv_scores}")
print(f"\nScore promedio: {cv_scores.mean():.4f}")
print(f"Desviación estándar: {cv_scores.std():.4f}")
print(f"Rango: [{cv_scores.min():.4f}, {cv_scores.max():.4f}]")

# Visualizar resultados de validación cruzada
plt.figure(figsize=(10, 6))
plt.bar(range(1, 6), cv_scores, alpha=0.7, color='steelblue', edgecolor='black')
plt.axhline(y=cv_scores.mean(), color='red', linestyle='--',
           linewidth=2, label=f'Promedio: {cv_scores.mean():.4f}')
plt.fill_between(range(1, 7),
                 cv_scores.mean() - cv_scores.std(),
                 cv_scores.mean() + cv_scores.std(),
                 alpha=0.2, color='red', label=f'±1 desviación estándar')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.title('Validación Cruzada 5-Fold - Bosque Aleatorio Optimizado')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.xticks(range(1, 6))
plt.tight_layout()
plt.show()


<a id="metricas"></a>
## 9. Métricas de Evaluación para Clasificación [⬆](#introduccion)

### 9.1 Matriz de Confusión

La matriz de confusión muestra cuántas muestras fueron clasificadas correctamente y dónde se cometieron errores.

#### Para Clasificación Multiclase (3 clases):
| Real \ Predicho | Funcional            | Marginal             | Fallado              |
|-----------------|----------------------|----------------------|----------------------|
| **Funcional**   | $\textsf{TP}_1$      | $\textsf{FN}_1→{}_2$ | $\textsf{FN}_1→{}_3$ |
| **Marginal**    | $\textsf{FN}_2→{}_1$ | $\textsf{TP}_2$      | $\textsf{FN}_2→{}_3$ |
| **Fallado**     | $\textsf{FN}_3→{}_1$ | $\textsf{FN}_3→{}_2$ | $\textsf{TP}_3$      |

Donde:
- **TP** (True Positive): Correctamente identificado como esa clase
- **FN** (False Negative): Clase A clasificada incorrectamente como clase B

### 9.2 Métricas Principales

#### Accuracy (Precisión Global)
- **Fórmula**: $(\textsf{TP}_1 + \textsf{TP}_2 + \textsf{TP}_3) / \textsf{Total}$
- **Interpretación**: Porcentaje de clasificaciones correctas
- **Cuándo usar**: Clases balanceadas

#### Precision por Clase
- **Fórmula**: $\textsf{Precision}_i = \textsf{TP}_i / (\textsf{TP}_i + \textsf{FP}_i)$
- **Interpretación**: De los predichos como clase $i$, ¿cuántos realmente son clase $i$?

#### Recall por Clase (Sensibilidad)
- **Fórmula**: $\textsf{Recall}_i = \textsf{TP}_i / (\textsf{TP}_i + \textsf{FN}_i)$
- **Interpretación**: De los realmente clase $i$, ¿cuántos detectamos?

#### F1-Score por Clase
- **Fórmula**: $\textsf{F1}_i = 2 \times (\textsf{Precision}_i \times \textsf{Recall}_i) / (\textsf{Precision}_i + \textsf{Recall}_i)$
- **Interpretación**: Balance entre Precision y Recall (media armónica)

#### F1-Score Macro (Promedio)
- **Fórmula**: $\textsf{F1}_\textsf{macro} = \frac{1}{n} \sum_{i=1}^{n} \textsf{F1}_i$
- **Interpretación**: Promedio del F1 de todas las clases (sin considerar desbalance)

### 9.3 ¿Qué Métrica Elegir?

| Escenario                  | Métrica Principal  | Razón                                    |
|----------------------------|--------------------|------------------------------------------|
| Clases balanceadas         | **Accuracy**       | Representa bien el rendimiento           |
| Clases desbalanceadas      | **F1-Score Macro** | No se sesga por clase mayoritaria        |
| Clase crítica (ej: fallos) | **Recall**         | Detectar todos los casos                 |
| Evitar falsos positivos    | **Precision**      | Asegurar que predicciones sean correctas |
| Balance general            | **F1-Score**       | Equilibrio entre Precision y Recall      |

---


<a id="comparacion"></a>
## 10. Comparación con Otros Algoritmos [⬆](#introduccion)

### Tabla Comparativa

| Algoritmo               | Interpretabilidad | Velocidad Entrenamiento | Manejo No Lineal | Robustez | Sobreajuste |
|-------------------------|-------------------|-------------------------|------------------|----------|-------------|
| **Árbol Decisión**      | Muy Alta          | Muy Alta                | Alta             | Media    | Alto        |
| **Random Forest**       | Alta              | Alta                    | Muy Alta         | Muy Alta | Bajo        |
| **SVM**                 | Media             | Alta                    | Muy Alta         | Alta     | Medio       |
| **Regresión Logística** | Muy Alta          | Muy Alta                | Baja             | Alta     | Bajo        |
| **K-NN**                | Alta              | Muy Alta                | Muy Alta         | Baja     | Alto        |
| **Redes Neuronales**    | Muy Baja          | Alta                    | Muy Alta         | Alta     | Alto        |

### Cuándo Usar Cada Algoritmo

#### Árbol de Decisión
- Necesitas explicabilidad y reglas claras  
- Dataset pequeño a mediano  
- Quieres identificar variables importantes  
- Necesitas modelo simple y rápido  

#### Random Forest
- Necesitas mejor precisión que árbol individual  
- Dataset mediano a grande  
- Relaciones no lineales complejas  
- Poder computacional disponible  
- Balance entre precisión e interpretabilidad  

#### SVM
- Dataset pequeño a mediano  
- Separaciones no lineales complejas  
- Alta dimensionalidad  

#### Regresión Logística
- Interpretabilidad importante  
- Dataset grande  
- Relaciones lineales o aproximadamente lineales  

#### K-NN
- Dataset pequeño  
- Necesitas modelo simple  
- Datos localmente estructurados  

#### Redes Neuronales
- Dataset muy grande  
- Relaciones muy complejas  
- Interpretabilidad no es crítica  

---

## 11. Ventajas y Desventajas - Resumen

### Árboles de Decisión

#### Ventajas
- Altamente interpretable
- No requiere normalización
- Maneja datos no lineales
- Identifica variables importantes
- Rápido de entrenar

#### Desventajas
- Propenso a sobreajuste
- Inestable (pequeños cambios cambian el árbol)
- Sesgo hacia variables con más valores

### Random Forest

#### Ventajas
- Mejor precisión que árbol individual
- Menos sobreajuste
- Robusto a outliers
- Mide importancia de características
- Maneja datos desbalanceados

#### Desventajas
- Menos interpretable que árbol individual
- Más lento y uso de memoria
- Parámetros adicionales a optimizar

---


<a id="resumen"></a>
## 12. Resumen y Conclusiones [⬆](#introduccion)

### 12.1 Puntos Clave para Recordar

#### Los 3 Conceptos Esenciales de Árboles
1. **Impureza (Gini/Entropía)**: Mide mezcla de clases en un nodo
2. **Ganancia de Información**: Guía qué atributo usar para dividir
3. **Poda**: Previene sobreajuste limitando complejidad

#### Los 3 Conceptos Esenciales de Random Forest
1. **Bootstrap Aggregating**: Cada árbol ve datos diferentes
2. **Selección Aleatoria de Características**: Reduce correlación entre árboles
3. **Votación Mayoritaria**: Decisión final por consenso

#### Los 3 Parámetros Críticos
1. **max_depth**: Controla profundidad (complejidad)
2. **n_estimators**: Número de árboles en el bosque
3. **min_samples_split**: Controla cuándo dividir un nodo

### 12.2 Flujo de Trabajo Recomendado

1. **Explorar datos**
   - Visualizar distribuciones
   - Analizar correlaciones
   - Entender desbalance de clases

2. **Árbol de decisión simple**
   - Entrenar árbol básico como baseline
   - Visualizar reglas
   - Analizar sobreajuste

3. **Random Forest**
   - Entrenar bosque con parámetros razonables
   - Analizar importancia de características
   - Comparar con árbol individual

4. **Optimización**
   - Grid Search con validación cruzada
   - Evaluar robustez
   - Seleccionar mejor modelo

5. **Evaluación final**
   - Matriz de confusión
   - Métricas apropiadas (Accuracy, F1, etc.)
   - Interpretar resultados

### 12.3 Mejores Prácticas

#### Hacer SIEMPRE
1. **Dividir datos** en entrenamiento y prueba (stratify si clases desbalanceadas)
2. **Visualizar árboles** para entender decisiones (profundidad limitada)
3. **Comprobar sobreajuste** comparando train vs test accuracy
4. **Usar Grid Search** con validación cruzada para optimizar
5. **Analizar importancia** de características
6. **Usar métricas apropiadas** según el problema (F1 para clases desbalanceadas)

#### Errores Comunes a Evitar
1. **No limitar profundidad** → Sobreajuste garantizado
2. **Usar datos de prueba durante entrenamiento** → Estimación sesgada
3. **Ignorar desbalance de clases** → Modelo sesgado hacia clase mayoritaria
4. **No validar con cross-validation** → Estimación no robusta
5. **Confiar solo en Accuracy** → Puede engañar con clases desbalanceadas

### 12.4 Aplicaciones en Ingeniería Electrónica

#### Casos de Uso Reales
1. **Control de Calidad**
   - Clasificación de componentes funcionales/fallados
   - Detección de defectos en fabricación

2. **Detección de Fallas**
   - Diagnóstico de circuitos
   - Predicción de fallas anticipadas

3. **Monitoreo de Procesos**
   - Clasificación de estados de operación
   - Detección de anomalías

4. **Calibración Automatizada**
   - Clasificación de niveles de ajuste necesarios
   - Selección de parámetros óptimos

### 12.5 Próximos Pasos

#### Para Practicar
1. Aplicar a dataset real de tu área
2. Experimentar con diferentes parámetros
3. Comparar con otros algoritmos (SVM, K-NN)
4. Implementar en casos industriales reales

#### Para Profundizar
- **Gradient Boosting**: XGBoost, LightGBM (ensembles avanzados)
- **Análisis de importancia**: SHAP values para explicabilidad
- **Árboles de regresión**: Aplicar conceptos a problemas continuos
- **Optimización bayesiana**: Alternativa más eficiente a Grid Search

---

## 13. Recursos y Referencias

### Documentación
- [Scikit-learn: Decision Trees](https://scikit-learn.org/stable/modules/tree.html)
- [Scikit-learn: Random Forest](https://scikit-learn.org/stable/modules/ensemble.html#random-forests)
- [Grid Search Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

### Libros Recomendados
- "An Introduction to Statistical Learning" - James, Witten, Hastie, Tibshirani
- "Pattern Recognition and Machine Learning" - Christopher Bishop
- "The Elements of Statistical Learning" - Hastie, Tibshirani, Friedman

### Cursos Online
- Machine Learning por Andrew Ng (Coursera)
- Applied Machine Learning (edX)

---

## 14. Apéndice: Ejemplo de Interpretación de Reglas

### Reglas de Decisión Ejemplo
```txt
SI temperatura <= 50°C:
    SI voltaje <= 5.2V:
        → Componente FUNCIONAL (95% confianza)
    SINO:
        SI corriente <= 110mA:
            → Componente DEGRADADO (78% confianza)
        SINO:
            → Componente CON FALLA (82% confianza)
SINO:
    SI temperatura <= 75°C:
        → Componente DEGRADADO (85% confianza)
    SINO:
        → Componente CON FALLA (92% confianza)
```

### Interpretación
- **Primera pregunta**: Temperatura es la variable más importante
- **Valores de umbral**: 50°C y 75°C son puntos críticos
- **Confianza**: Probabilidad de clasificación correcta

---

## ¿Preguntas?

*Tiempo restante para preguntas y discusión*

---

## ¡Gracias por su atención!
